In [161]:
!pip install joblib

In [301]:
import numpy as np
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter(action='ignore')

X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')


# 전처리

In [302]:
x_data=X_train.append(X_test)
y_data=y_train.append(y_test)
x_data=x_data.iloc[:,1:]
y_data=y_data.iloc[:,1:]
x_data

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0
...,...,...,...,...,...,...,...,...,...
2667,audi,A7,2015,Semi-Auto,21100,Petrol,325.0,29.7,4.0
2668,merc,CLS Class,2015,Automatic,60972,Diesel,160.0,52.3,3.0
2669,ford,Puma,2020,Manual,4111,Petrol,145.0,50.4,1.0
2670,merc,CLA Class,2016,Automatic,25726,Petrol,200.0,41.5,2.0


In [279]:
x_data=pd.get_dummies(x_data)

In [280]:
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

## 라벨인코딩...

In [169]:
# le=LabelEncoder()
# le2=LabelEncoder()

In [170]:
X_train

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0
...,...,...,...,...,...,...,...,...,...,...
4955,16898,merc,GL Class,2015,Automatic,24314,Diesel,125.0,56.6,2.1
4956,14416,bmw,6 Series,2017,Automatic,18000,Diesel,145.0,51.4,3.0
4957,15453,vw,CC,2015,Manual,84932,Diesel,30.0,60.1,2.0
4958,14666,audi,A7,2017,Semi-Auto,30150,Diesel,145.0,62.8,3.0


In [171]:
# le.fit(X_train.brand.unique())
# le2.fit(X_train.model.unique())
# X_train['brand']=le.transform(X_train['brand'])
# X_train['model']=le2.transform(X_train['model'])
# X_test['brand']=le.transform(X_test['brand'])
# X_test['model']=le2.transform(X_test['model'])
# ['transmission']

# 표준화

In [281]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaled = scaler.fit_transform(x_data)

X_train, X_test,y_train,y_test=train_test_split(scaled,y_data,test_size=0.3)

# 릿지

In [282]:
ridge = Ridge()


ridge.fit(X_train,y_train)
rid_pred=ridge.predict(X_test)
neg_mse_scores = cross_val_score(ridge, X_test, y_test, scoring="neg_mean_squared_error", cv = 5)
rmse_scores  = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
print(' 5 folds 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 3))
print(' 5 folds 의 개별 RMSE scores : ', np.round(rmse_scores,3))
print(' 5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))
print('score : {0:.3f}'.format(r2_score(y_test, rid_pred)))

 5 folds 의 개별 Negative MSE scores:  [-36267832.69  -26852069.877 -50729396.516 -34773778.004 -33985261.119]
 5 folds 의 개별 RMSE scores :  [6022.278 5181.898 7122.457 5896.93  5829.688]
 5 folds 의 평균 RMSE : 6010.650 
score : 0.884


In [284]:
alphas = [0,0.1,10, 100]

for alpha in alphas :
    ridge = Ridge(alpha = alpha)
    ridge.fit(X_train,y_train)
    neg_mse_scores = cross_val_score(ridge, X_test, y_test, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    print('alpha {0} 일 때 5 folds 의 평균 RMSE : {1:.3f} '.format(alpha,avg_rmse))
    print('score : {0:.3f}'.format(r2_score(y_test, rid_pred)))

alpha 0 일 때 5 folds 의 평균 RMSE : 724398716363738368.000 
score : 0.884
alpha 0.1 일 때 5 folds 의 평균 RMSE : 6012.861 
score : 0.884
alpha 10 일 때 5 folds 의 평균 RMSE : 5998.826 
score : 0.884
alpha 100 일 때 5 folds 의 평균 RMSE : 6046.160 
score : 0.884


In [287]:
#모델 파일로 저장
joblib.dump(ridge, 'car_model.pkl')
#저장한 모델 로드
model = joblib.load('car_model.pkl')